In [94]:
import pandas as pd
import numpy as np
!pip install pulp
import pulp
import copy
import random
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pathlib import Path
import warnings
warnings.filterwarnings("ignore")

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: C:\Users\JHONATAN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [95]:
def opt_port(df_sample,epsilon):
    epsilon = epsilon
    X_tr = df_sample.copy(deep=True)

    mto_van_opt = df_sample[df_sample.flg_max_vane==1]['mto_e'].sum()
    mto_opt_port = mto_van_opt*epsilon

    idcs = list(df_sample['codclavecic'].unique())
    df_sample.set_index(keys=['codclavecic','escenario'], inplace=True)

    esc_opt = pulp.LpVariable.dicts("esc_opt",
                                    ((idc, escenario) for idc, escenario in df_sample.index),
                                    cat='Binary')

    # Definición del Modelo
    model_opt = pulp.LpProblem("Optimización_del_Van_Esperado", pulp.LpMaximize)

    # Función Objetivo
    model_opt += pulp.lpSum([esc_opt[idc, escenario] * df_sample.loc[(idc, escenario), 'van_e'] for idc, escenario in df_sample.index]) 

    # Restricciones
    for idc in idcs:
        escenarios = X_tr['escenario'][X_tr['codclavecic']==idc]
        model_opt += pulp.lpSum(esc_opt[(idc, esc)] for esc in escenarios) == 1

    model_opt += pulp.lpSum([esc_opt[idc, escenario] * df_sample.loc[(idc, escenario), 'mto_e'] for idc, escenario in df_sample.index]) >= mto_opt_port

    model_opt.solve()


    print(pulp.LpStatus[model_opt.status])
    print(model_opt.objective.value())


    output = []
    for idc, esc in esc_opt:
        var_output = {
            'idc': idc, 
            'escenario': esc, 
            'binario': esc_opt[(idc, esc)].varValue, 
            'value':df_sample.loc[(idc,esc),'van_e'] }
        
        output.append(var_output) 

    output_df = pd.DataFrame.from_records(output)
    return output_df

In [3]:
route_bases = Path('.').resolve() / 'data' / 'bases'
route_output = Path('.').resolve() / 'data' / 'optimizacion'

input_filename = "wtp_escenarios_ind_20231229.csv"
output_filename_5 = f"salida_opt_port_campania_{(datetime.now() + relativedelta(months=1)).strftime('%Y%m')}_5_new.csv"
output_filename_10 = f"salida_opt_port_campania_{(datetime.now() + relativedelta(months=1)).strftime('%Y%m')}_10_new.csv"


output_filename_5_final, extension = output_filename_5.rsplit('.', 1)  
output_filename_5_final = output_filename_5_final.replace("new", "escenario_new")  
output_filename_5_final = f"{output_filename_5_final}.{extension}" 

output_filename_10_final, extension = output_filename_10.rsplit('.', 1)  
output_filename_10_final = output_filename_10_final.replace("new", "escenario_new")  
output_filename_10_final = f"{output_filename_10_final}.{extension}" 


full_filename_input = route_bases / input_filename
full_filename_output_5 = route_output/ output_filename_5
full_filename_output_10 = route_output/ output_filename_10
full_filename_output_5_final = route_output/ output_filename_5_final

full_filename_output_10_final = route_output/ output_filename_10_final

In [114]:
df = pd.read_csv( 'C:/Users/JHONATAN/Documents/BCP  practicas verano/CEF_ Credito_Efectivo_Personas/10000CEN_CEF_ADAPTACIONBase_para_probar_CENOPT.csv',sep=',')
df.columns = df.columns.str.lower()
df.columns

Index(['codmes', 'seg_wtp_cef', 'numscoreriesgo', 'codsubsegmento',
       'codclavecic', 'llave', 'mtofinalofertadosol', 'rng_prob', 'num_sen',
       'iter', 'tasa_propuesta', 'prob_adj', 'prob', 'van', 'van_estimado',
       'mto_estimado', 'ec_efect_i', 'b0', 'b1', 'prob_adj_ef',
       'flg_subsegmento_consumo'],
      dtype='object')

In [115]:
df = df.rename(columns={'iter': 'escenario',
                        'tasa_propuesta':'tea',
                        'van_estimado':'van_e',
                        'mto_estimado':'mto_e',
                        'prob_adj':'tir',
                        'prob_adj_ef':'efec',
                        })

In [116]:
df.columns

Index(['codmes', 'seg_wtp_cef', 'numscoreriesgo', 'codsubsegmento',
       'codclavecic', 'llave', 'mtofinalofertadosol', 'rng_prob', 'num_sen',
       'escenario', 'tea', 'tir', 'prob', 'van', 'van_e', 'mto_e',
       'ec_efect_i', 'b0', 'b1', 'efec', 'flg_subsegmento_consumo'],
      dtype='object')

In [117]:
df.head(3)

,codmes,seg_wtp_cef,numscoreriesgo,codsubsegmento,codclavecic,llave,mtofinalofertadosol,rng_prob,num_sen,escenario,...,tir,prob,van,van_e,mto_e,ec_efect_i,b0,b1,efec,flg_subsegmento_consumo
0,202401,1,336,M1N,93457,93457-LD-CRE,9700.0,2.[Baja],-2.57,-8,...,0.271264,0.834083,392.023845,1.287477,31.856545,0.012107,0.020706,-0.024639,0.003284,ARMANDO
1,202401,1,336,M1N,93457,93457-LD-CRE,9700.0,2.[Baja],-2.57,0,...,0.251427,0.819364,538.704177,1.506331,27.123263,0.011121,0.020706,-0.024639,0.002796,ARMANDO
2,202401,1,336,M1N,93457,93457-LD-CRE,9700.0,2.[Baja],-2.57,5,...,0.239061,0.809263,631.927082,1.583317,24.303714,0.010481,0.020706,-0.024639,0.002506,ARMANDO


In [118]:
df= df[['codclavecic','escenario','tea','van_e', 'mto_e','tir','efec']].sort_values(by=['codclavecic','escenario'])

In [120]:
idx_max = df.groupby('codclavecic')['van_e'].idxmax()

df.loc[idx_max, 'flg_max_vane'] = 1
df['flg_max_vane']=df['flg_max_vane'].fillna(0)


In [121]:
df.head(20)

,codclavecic,escenario,tea,van_e,mto_e,tir,efec,flg_max_vane
0,93457,-8,0.349,1.287477,31.856545,0.271264,0.003284,0.0
8,93457,-7,0.355,1.328898,31.115202,0.268227,0.003208,0.0
11,93457,-6,0.359,1.354742,30.627195,0.266214,0.003157,0.0
3,93457,-5,0.365,1.390923,29.904459,0.263213,0.003083,0.0
14,93457,-4,0.369,1.413360,29.428782,0.261224,0.003034,0.0
16,93457,-3,0.375,1.444556,28.724429,0.258259,0.002961,0.0
13,93457,-2,0.379,1.463752,28.260933,0.256295,0.002913,0.0
4,93457,-1,0.385,1.490211,27.574734,0.253367,0.002843,0.0
1,93457,0,0.389,1.506331,27.123263,0.251427,0.002796,0.0
5,93457,1,0.395,1.528296,26.454985,0.248536,0.002727,0.0


In [74]:
data = {'columna_a': ['Juan', 'Juan', 'Juan', 'Pedro', 'Pedro', 'Pedro', 'Marco', 'Marco','Marco'],
        'columna_b': [1, 2, 3, 1, 2, 3, 1, 2, 3],
        }
x=pd.DataFrame(data)

x['columna_c'] = 0
idx_max = x.groupby('columna_a')['columna_b'].idxmax()

# Asignar "1" a la nueva columna_c para las filas identificadas por idx_max
x.loc[idx_max, 'columna_c'] = 1
x

,columna_a,columna_b,columna_c
0,Juan,1,0
1,Juan,2,0
2,Juan,3,1
3,Pedro,1,0
4,Pedro,2,0
5,Pedro,3,1
6,Marco,1,0
7,Marco,2,0
8,Marco,3,1


In [67]:
df.head(18)

,codclavecic,escenario,tea,van_e,mto_e,tir,efec
0,93457,-8,0.349,1.287477,31.856545,0.271264,0.003284
8,93457,-7,0.355,1.328898,31.115202,0.268227,0.003208
11,93457,-6,0.359,1.354742,30.627195,0.266214,0.003157
3,93457,-5,0.365,1.390923,29.904459,0.263213,0.003083
14,93457,-4,0.369,1.413360,29.428782,0.261224,0.003034
16,93457,-3,0.375,1.444556,28.724429,0.258259,0.002961
13,93457,-2,0.379,1.463752,28.260933,0.256295,0.002913
4,93457,-1,0.385,1.490211,27.574734,0.253367,0.002843
1,93457,0,0.389,1.506331,27.123263,0.251427,0.002796
5,93457,1,0.395,1.528296,26.454985,0.248536,0.002727


In [68]:
# Load DataFrame

#df = pd.read_csv( 'C:\Users\JHONATAN\Documents\BCP  practicas verano\CEF_ Credito_Efectivo_Personas\10000CEN_CEF_ADAPTACIONBase_para_probar_CENOPT.csv',sep=',')

# Lista de Clientes
clientes = set(df['codclavecic'])

# Número de samples que deseas generar
samples_num = 10

# Tamaño de las submuestras
sample_size = int(len(clientes)/samples_num)

# Lista de Dataframes
df_list = []
output_list = []
tiempo = []

porc = 1.05

print("Clientes totales del DataFrame:",len(df.codclavecic.unique()),"\n")

# generar las submuestras sin repetir elementos
for i in range(1,samples_num+1):
    if i<samples_num: 
        clientes_sample = random.sample(clientes, sample_size)
        df_sample = df[df.codclavecic.isin(clientes_sample)]
        df_sample["sample_num"] = i 
        df = df.drop(df_sample.index)

        print("Clientes del DataFrame del grupo",i,":",len(df_sample.codclavecic.unique()))
        print("Clientes que quedan en el DataFrame:", len(df.codclavecic.unique()))
       
    
        df_list.append(df_sample)
        clientes = clientes - set(clientes_sample)

        start_time = datetime.now() 

        output_df = opt_port(df_sample,porc)
        output_list.append(output_df)
        
        time_elapsed = datetime.now() - start_time  
        tiempo.append(time_elapsed)
        print('Time elapsed (hh:mm:ss.ms) {time_elapsed}, grupo:{grupo}'.format(time_elapsed=time_elapsed, grupo=i),"\n")
        
    else:
        df_sample = df.copy()
        df_sample["sample_num"] = i    
        df_list.append(df_sample)
        
        print("Clientes del último sample:",len(df_sample.codclavecic.unique()))
        
        start_time = datetime.now() 
       
        output_df = opt_port(df_sample,porc)
        output_list.append(output_df)

        time_elapsed = datetime.now() - start_time  
        tiempo.append(time_elapsed)
        print('Time elapsed (hh:mm:ss.ms) {time_elapsed}, grupo:{grupo}'.format(time_elapsed=time_elapsed, grupo=i),"\n")
        
      
df_tot = pd.concat(df_list)   
df_tot_opt = pd.concat(output_list)

Clientes totales del DataFrame: 590 



TypeError: Population must be a sequence.  For dicts or sets, use sorted(d).

In [5]:
df_tot.shape

(259177, 9)

In [6]:
df_tot_opt.to_csv(full_filename_output_5,index=False)

In [7]:
df_tot_opt = pd.read_csv(full_filename_output_5)

#Salida del WTP Individual Escenarios
df_wtp_esc = pd.read_csv(full_filename_input)

#Salida de la Optimización del Portafolio tomando los leads por grupo
df_opt_port_group = df_tot_opt[df_tot_opt.binario==1][['idc','escenario']].rename(columns={'idc':'codclavecic'})

#Se le pega la TEA Opt por grupo de acuerdo al escenario
df_tea_opt_port_by_group = pd.merge(df_opt_port_group, df_wtp_esc[['codclavecic','escenario','tea', 'cluster','tir']], on=['codclavecic','escenario'], how='left')

df_tea_opt_port_by_group.to_csv(full_filename_output_5_final, index=False)

In [8]:
df_tea_opt_port_by_group.shape

(18579, 5)

In [14]:
# Load DataFrame
df = pd.read_csv(full_filename_input)

# Lista de Clientes
clientes = set(df['codclavecic'])

# Número de samples que deseas generar
samples_num = 10

# Tamaño de las submuestras
sample_size = int(len(clientes)/samples_num)

# Lista de Dataframes
df_list = []
output_list = []

porc = 1.061

print("Clientes totales del DataFrame:",len(df.codclavecic.unique()),"\n")

# generar las submuestras sin repetir elementos
for i in range(1,samples_num+1):
    if i<samples_num:
        clientes_sample = random.sample(clientes, sample_size)
        df_sample = df[df.codclavecic.isin(clientes_sample)]
        df_sample["sample_num"] = i 
        df = df.drop(df_sample.index)

        print("Clientes del DataFrame del grupo",i,":",len(df_sample.codclavecic.unique()))
        print("Clientes que quedan en el DataFrame:", len(df.codclavecic.unique()))
       
    
        df_list.append(df_sample)
        clientes = clientes - set(clientes_sample)

        start_time = datetime.now() 

        output_df = opt_port(df_sample,porc)
        output_list.append(output_df)

        time_elapsed = datetime.now() - start_time  
        print('Time elapsed (hh:mm:ss.ms) {time_elapsed}, grupo:{grupo}'.format(time_elapsed=time_elapsed, grupo=i),"\n")
        
    else:
        df_sample = df.copy()
        df_sample["sample_num"] = i    
        df_list.append(df_sample)
        
        print("Clientes del último sample:",len(df_sample.codclavecic.unique()))
        
        start_time = datetime.now() 
       
        output_df = opt_port(df_sample,porc)
        output_list.append(output_df)

        time_elapsed = datetime.now() - start_time  
        print('Time elapsed (hh:mm:ss.ms) {time_elapsed}, grupo:{grupo}'.format(time_elapsed=time_elapsed, grupo=i),"\n")
        
      
df_tot_10 = pd.concat(df_list)   
df_tot_opt_10 = pd.concat(output_list) 

Clientes totales del DataFrame: 18579 

Clientes del DataFrame del grupo 1 : 1857
Clientes que quedan en el DataFrame: 16722
Optimal
137064.396979098
Time elapsed (hh:mm:ss.ms) 0:00:27.943870, grupo:1 

Clientes del DataFrame del grupo 2 : 1857
Clientes que quedan en el DataFrame: 14865
Optimal
133502.8731850735
Time elapsed (hh:mm:ss.ms) 0:00:28.597663, grupo:2 

Clientes del DataFrame del grupo 3 : 1857
Clientes que quedan en el DataFrame: 13008
Optimal
146382.3465535692
Time elapsed (hh:mm:ss.ms) 0:00:14.047463, grupo:3 

Clientes del DataFrame del grupo 4 : 1857
Clientes que quedan en el DataFrame: 11151
Optimal
143269.92845842056
Time elapsed (hh:mm:ss.ms) 0:00:19.788370, grupo:4 

Clientes del DataFrame del grupo 5 : 1857
Clientes que quedan en el DataFrame: 9294
Optimal
128798.93850335418
Time elapsed (hh:mm:ss.ms) 0:00:09.821486, grupo:5 

Clientes del DataFrame del grupo 6 : 1857
Clientes que quedan en el DataFrame: 7437
Optimal
146767.73621654997
Time elapsed (hh:mm:ss.ms) 0:

In [15]:
df_tot_opt_10.to_csv(full_filename_output_10,index=False)

In [16]:
df_tot_opt_10.shape

(259177, 4)

In [17]:
df_tot_opt_10 = pd.read_csv(full_filename_output_10)

#Salida del WTP Individual Escenarios
df_wtp_esc = pd.read_csv(full_filename_input)

#Salida de la Optimización del Portafolio tomando los leads por grupo
df_opt_port_group = df_tot_opt_10[df_tot_opt_10.binario==1][['idc','escenario']].rename(columns={'idc':'codclavecic'})

#Se le pega la TEA Opt por grupo de acuerdo al escenario
df_tea_opt_port_by_group = pd.merge(df_opt_port_group, df_wtp_esc[['codclavecic','escenario','tea', 'cluster','tir']], on=['codclavecic','escenario'], how='left')

df_tea_opt_port_by_group.to_csv(full_filename_output_10_final, index=False)

In [18]:
df_tea_opt_port_by_group.shape

(18579, 5)